In [23]:
from glob import glob
import sys, time, os, asyncio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline

from lsst_efd_client import EfdClient, resample, merge_packed_time_series, rendezvous_dataframes
from lsst.ts.idl.enums import MTM1M3

from scipy.signal import find_peaks
from scipy.interpolate import interp1d
import astropy.units as u

In [24]:
events=pd.read_csv("/home/p/pferguso/u/projects/m1m3/hp_oscillations_during_slew/data/230622_combined_events.csv")

In [25]:
events=events.sort_values("times", ascending=False).reset_index(drop=True)

In [26]:
def link_from_date(timestamp):
    
    begin_timestamp=(Time(timestamp, format="unix") - TimeDelta(3, format='sec')).iso
    end_timestamp=(Time(timestamp, format="unix") + TimeDelta(3, format='sec')).iso
    
    begin_day=begin_timestamp[:10]
    begin_hour=begin_timestamp[11:13]
    begin_minute=begin_timestamp[14:16]
    begin_second=begin_timestamp[17:]
    
    end_day=end_timestamp[:10]
    end_hour=end_timestamp[11:13]
    end_minute=end_timestamp[14:16]
    end_second=end_timestamp[17:]
    
    url="https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/"
    url+="207?refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D="
    url+=f"raw&lower={begin_day}T{begin_hour}%3A{begin_minute}%3A{begin_second}Z"
    url+=f'&upper={end_day}T{end_hour}%3A{end_minute}%3A{end_second}Z'
    #url+=f"&zoomedLower={begin_day}T{begin_hour}%3A{begin_minute}%3A{begin_second}Z&zoomedUpper=2023-05-30T10%3A08%3A02.962Z"
    return url

In [27]:
title_str="| event number  | Time  |  elevation |  elevation velocity | hardpoint/ims dashboard  |\n|---|---|---|---|---|\n"


In [28]:
row_str=""
for j,i in enumerate((range(len(events)))):
    #if events["group"][i] < 3:
    row_str+=f"| {len(events)-j}  | {events['iso_time'][i]}  | "
    row_str+=f"{events['elevation_position'][i]:0.2f}  | {events['elevation_velocity'][i]:0.2f} " 
    row_str+='| <a href="'
    row_str+=f'{link_from_date(events["times"][i]-2)}'
    row_str+='" target="_blank">link to dashboard</a>|\n'

In [29]:
print(title_str+row_str)

| event number  | Time  |  elevation |  elevation velocity | hardpoint/ims dashboard  |
|---|---|---|---|---|
| 126  | 2023-06-23 20:20:14.944  | 64.21  | 0.05 | <a href="https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/207?refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=raw&lower=2023-06-23T20%3A20%3A09.944Z&upper=2023-06-23T20%3A20%3A15.944Z" target="_blank">link to dashboard</a>|
| 125  | 2023-06-23 20:15:10.688  | 48.99  | 0.05 | <a href="https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/207?refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=raw&lower=2023-06-23T20%3A15%3A05.688Z&upper=2023-06-23T20%3A15%3A11.688Z" target="_blank">link to dashboard</a>|
| 124  | 2023-06-23 17:12:30.626  | 12.56  | -0.05 | <a href="https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/207?refresh=Paused&tempVars%5BDownsample%5D=Default&tempVars%5BFunction%5D=raw&lower=2023-06-23T17%3A12%3A25.626Z&upper=2023-06-23T17%3A12%3A3